## VGG16 pre-trained model

In order to explore tranfer learning we are going to use a VGG16 model pre-trained with the ImageNet dataset. The model we are going to use are the ones provided by keras. We are not going to used the pre-trained top layers, instead we are going to define our small Fully Connected Network. This FCN is going to be trained to try to use the features calculated by the pre-trained layers with the new dataset of "cats and dogs"

## Common configuration

In [1]:
IMAGE_SIZE = (360,404) # The dimensions to which all images found will be resized.
BATCH_SIZE = 32
NUMBER_EPOCHS = 8

TENSORBOARD_DIRECTORY = "../logs/simple_model/tensorboard"
TRAIN_DIRECTORY = "../data/train/"
VALID_DIRECTORY = "../data/valid/"

NUMBER_TRAIN_SAMPLES = 20000
NUMBER_VALIDATION_SAMPLES = 5000

PRECOMPUTED_DIRECTORY = "../precomputed/vgg16/"

Check that we are using the GPU:

In [ ]:
from tensorflow.python.client import device_lib
def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']
    
get_available_gpus()

## Model

### Model definition

In [2]:
from keras.applications.vgg16 import VGG16

# create the base pre-trained model
base_model = VGG16(weights='imagenet', include_top=False)

Using TensorFlow backend.


### Base Model arquitecture

We have the following model arquitecture:

In [3]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

### Calculating the precomputed features

#### Calculate and save validation features

In [8]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

datagen = ImageDataGenerator()

generator = datagen.flow_from_directory(
        VALID_DIRECTORY,
        target_size=(IMAGE_SIZE[0], IMAGE_SIZE[1]),
        batch_size=1,
        class_mode=None,  # this means our generator will only yield batches of data, no labels
        shuffle=False)  # our data will be in order
    
validation_features = np.concatenate([base_model.predict(next(generator)) for i in range(NUMBER_VALIDATION_SAMPLES)])

Found 5000 images belonging to 2 classes.


KeyboardInterrupt: 

In [ ]:
validation_features.shape

In [ ]:
folder = PRECOMPUTED_DIRECTORY
print(folder)
%mkdir $folder

file_path = folder + '/validation_features.npy' #fixme: use os lib for multi os support
file = open(file_path, 'wb')
np.save(file, validation_features)
print('Saving Validation images to {}'.format(file))

#### Calculate and save training features

In [ ]:
datagen = ImageDataGenerator()

generator = datagen.flow_from_directory(
        TRAIN_DIRECTORY,
        target_size=(IMAGE_SIZE[0], IMAGE_SIZE[1]),
        batch_size=1,
        class_mode=None,  # this means our generator will only yield batches of data, no labels
        shuffle=False)  # our data will be in order
    
train_features = np.concatenate([base_model.predict(next(generator)) for i in range(NUMBER_TRAIN_SAMPLES)])

In [ ]:
train_features.shape

In [ ]:
folder = PRECOMPUTED_DIRECTORY
print(folder)
%mkdir $folder

file_path = folder + '/train_features.npy' #fixme: use os lib for multi os support
file = open(file_path, 'wb')
np.save(file, train_features)
print('Saving Validation images to {}'.format(file))

#### Calculate and save test data

In [ ]:
datagen = ImageDataGenerator()

generator = datagen.flow_from_directory(
        TEST_DIRECTORY,
        target_size=(IMAGE_SIZE[0], IMAGE_SIZE[1]),
        batch_size=1,
        class_mode=None,  # this means our generator will only yield batches of data, no labels
        shuffle=False)  # our data will be in order
    
test_features = np.concatenate([base_model.predict(next(generator)) for i in range(1000)])

In [ ]:
test_features.shape

In [ ]:
folder = PRECOMPUTED_DIRECTORY
print(folder)
%mkdir $folder

file_path = folder + '/test_features.npy' #fixme: use os lib for multi os support
file = open(file_path, 'wb')
np.save(file, test_features)
print('Saving Validation images to {}'.format(file))

#### Filenames and labels

Save and load classes and filenames:

In [ ]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_all_classes()

In [ ]:
import pickle

file = PRECOMPUTED_DIRECTORY + '/classes_and_filenames.dat'

# Saving the objects:
with open(file, 'wb') as file:  # Python 2: open(..., 'w')
    pickle.dump([val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames], file)
    

### Keras callbacks

We are going to define two callbacks that are going to be called in the training. EarlyStopping to stop the training if its not getting better. And a tensorboard callback to log information to be used by tensorboard.

In [11]:
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard

# Early stop in case of getting worse
early_stop = EarlyStopping(monitor = 'val_loss', patience = 3, verbose = 0)

#TensorBoard
# run tensorboard with tensorboard --logdir=/full_path_to_your_logs
tensorboard_path = TENSORBOARD_DIRECTORY
tensorboard_logger = TensorBoard(log_dir=tensorboard_path, histogram_freq=0, write_graph=False, write_images=False)
print('Logging basic info to be used by TensorBoard to {}. To see this log run:'.format(tensorboard_path))
print('tensorboard --logdir={}'.format(tensorboard_path))

callbacks = [early_stop, tensorboard_logger]

Logging basic info to be used by TensorBoard to ../logs/simple_model/tensorboard. To see this log run:
tensorboard --logdir=../logs/simple_model/tensorboard


### Model Optimizer

In [12]:
OPTIMIZER_LEARNING_RATE = 1e-2
OPTIMIZER_DECAY = 1e-4  # LearningRate = LearningRate * 1/(1 + decay * epoch)
OPTIMIZER_MOMENTUM = 0.89
OPTIMIZER_NESTEROV_ENABLED = False

In [13]:
from keras.optimizers import SGD

optimizer = SGD(lr=OPTIMIZER_LEARNING_RATE, 
          decay=OPTIMIZER_DECAY, 
          momentum=OPTIMIZER_MOMENTUM, 
          nesterov=OPTIMIZER_NESTEROV_ENABLED)

### Compile the model

In [15]:
model.compile(loss='categorical_crossentropy', 
              optimizer=optimizer, \
              metrics=["accuracy"])

## Training 

### Train data generator

In [21]:
from keras.preprocessing.image import ImageDataGenerator

## train generator with shuffle but no data augmentation
train_datagen = ImageDataGenerator(rescale = 1./255)

train_batch_generator =  train_datagen.flow_from_directory(TRAIN_DIRECTORY, 
                                                 target_size = IMAGE_SIZE,
                                                 class_mode = 'categorical', 
                                                 batch_size = BATCH_SIZE)

Found 20000 images belonging to 2 classes.


### Validation data generator

In [23]:
from keras.preprocessing.image import ImageDataGenerator

## train generator with shuffle but no data augmentation
validation_datagen = ImageDataGenerator(rescale = 1./255)

valid_batch_generator =  validation_datagen.flow_from_directory(VALID_DIRECTORY, 
                                                 target_size = IMAGE_SIZE,
                                                 class_mode = 'categorical', 
                                                 batch_size = BATCH_SIZE)

Found 5000 images belonging to 2 classes.


### Model fitting

In [ ]:
# fine-tune the model
hist = model.fit_generator(
        train_batch_generator,
        steps_per_epoch=NUMBER_TRAIN_SAMPLES/BATCH_SIZE,
        epochs=NUMBER_EPOCHS,  # epochs: Integer, total number of iterations on the data.
        validation_data=valid_batch_generator,
        validation_steps=NUMBER_VALIDATION_SAMPLES/BATCH_SIZE,
        callbacks=callbacks,
        verbose=2)

### Training plots

In [ ]:
import matplotlib.pyplot as plt

# summarize history for accuracy
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.plot(hist.history['acc']); plt.plot(hist.history['val_acc']);
plt.title('model accuracy'); plt.ylabel('accuracy');
plt.xlabel('epoch'); plt.legend(['train', 'valid'], loc='upper left');

# summarize history for loss
plt.subplot(1, 2, 2)
plt.plot(hist.history['loss']); plt.plot(hist.history['val_loss']);
plt.title('model loss'); plt.ylabel('loss');
plt.xlabel('epoch'); plt.legend(['train', 'valid'], loc='upper left');
plt.show()

## Plot a few examples

## Evaluate the model